# Telecommunication_user_churn_classifier


1. Data Mining



In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
#print("Libraries imported successfully.....")
#Importing data
df = pd.read_csv("telecom_customer_churn.csv")

#print("Data imported successfully.....")


In [ ]:
# Remove the columns "Customer Status", "Churn Category", and "Churn Reason"
columns_to_remove = ["Customer Status", "Churn Category", "Churn Reason"]
df_train = df.drop(columns=columns_to_remove)

# Take out 30% of the DataFrame as the test set
test_size = 0.3
df_test = df_train.sample(frac=test_size, random_state=42)
df = df.drop(index=df_test.index)


2. Data Cleaning and EDA


In [ ]:
#Separate numerical and categorical columns

from preprolib import myfunctions
num_cols = []
cat_cols = []

ignore_list = ['Zip Code', 'Longitude', 'Latitude', 
                'Customer ID', 'Churn Category', 
                'Churn Reason', 'Customer Status', 'City']

myfunctions.cat_or_num(df, ignore_list, num_cols, cat_cols)

label = 'Customer Status'


2.1 Exploratory Data Analysis


2.1.1 Numerical data


Distribution


In [ ]:
##num_cols.remove

remove_num = ['Age', 'Avg Monthly Long Distance Charges',  'Avg Monthly GB Download', 'Monthly Charge']

num_cols = [x for x in num_cols if x not in remove_num]


##cat_cols.remove

remove_cat = ['Gender', 'Married',  'Multiple Lines']

cat_cols = [x for x in cat_cols if x not in remove_cat]


In [ ]:
for col in num_cols:
    plt.figure(figsize=(8, 6))
    plt.hist(df[col], bins=20, edgecolor='k', alpha=0.7)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

Univariate analysis

In [ ]:
# Box plots for numerical data
for column in num_cols:
    plt.figure()  # Create a new figure for each box plot
    sns.boxplot(x='Customer Status', y=column, data=df)
    plt.title(f'Box Plot for {column} against {"Customer Status"}')
    plt.ylabel(column)
    plt.xlabel('Target')
    plt.show()

num_cols


2.1.2 Categorical data

  



Disstribution

In [ ]:
for col in cat_cols:
    plt.figure(figsize=(8, 6))
    cat_counts = df[col].value_counts()
    cat_counts.plot(kind='bar', edgecolor='k', alpha=0.7)
    plt.title(f'Bar Plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define interactive widgets
feature_dropdown = widgets.Dropdown(
    options=cat_cols,
    description='Feature:'
)

status1_dropdown = widgets.Dropdown(
    options=df['Customer Status'].unique(),
    description='Status 1:'
)

status2_dropdown = widgets.Dropdown(
    options=df['Customer Status'].unique(),
    description='Status 2:'
)

# Create an output widget for displaying the visualization
output = widgets.Output()

# Function to create and display pie charts
def create_and_display_pie_charts(feature, status1, status2):
    with output:
        clear_output(wait=True)  # Clear the previous output
        filtered_df = df[(df['Customer Status'].isin([status1, status2]))]

        # Create a subplot for each category within the selected feature
        unique_categories = filtered_df[feature].unique()
        num_categories = len(unique_categories)

        fig, axes = plt.subplots(1, num_categories, figsize=(15, 5))

        for i, category in enumerate(unique_categories):
            category_counts = filtered_df[filtered_df[feature] == category]['Customer Status'].value_counts()
            ax = axes[i]
            ax.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=90)
            ax.set_title(category)
            ax.axis('equal')

        plt.show()

# Define the interactive function
def update_pie_charts(change):
    feature = feature_dropdown.value
    status1 = status1_dropdown.value
    status2 = status2_dropdown.value

    create_and_display_pie_charts(feature, status1, status2)

# Set up the interactivity
feature_dropdown.observe(update_pie_charts, names='value')
status1_dropdown.observe(update_pie_charts, names='value')
status2_dropdown.observe(update_pie_charts, names='value')

# Display the widgets and output area
display(widgets.VBox([feature_dropdown, status1_dropdown, status2_dropdown]))
display(output)

# Display initial pie charts based on default selections
create_and_display_pie_charts(cat_cols[0], df['Customer Status'].unique()[0], df['Customer Status'].unique()[1])


2.2 Data preprocessing


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Define the features and label
features = cat_cols + num_cols
label = 'Customer Status'

# Convert the label column to ordinal categories
label_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
y = label_encoder.fit_transform(df[label].values.reshape(-1, 1))

# Function to encode based on the percentage of churn per category
def custom_encode(column, churn_column):
    churn_percentage = df.groupby(column)[churn_column].mean()
    return churn_percentage.rank().fillna(0).astype(int)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], y, test_size=0.3, random_state=0)

# Define a pipeline for numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define a pipeline for categorical columns
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Create a ColumnTransformer to apply the pipeline to the numeric and categorical columns
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])

# Fit the preprocessor to the training data and transform both the training and test data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print('Training Set: %d, Test Set: %d \n' % (len(X_train), len(X_test)))

# Print the transformed DataFrames
print("X_train_Transformed:\n", X_train_transformed)
print("\nX_test_Transformed:\n", X_test_transformed)


# Create a dictionary to map encoded variables to original labels
label_mapping = {}
for i, label in enumerate(label_encoder.categories_[0]):
    label_mapping[i] = label

# Print the dictionary
print("Encoded to Original Label Mapping:")
print(label_mapping)


3. Model Selection


In [ ]:
#Import classifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=1).fit(X_train_transformed, y_train)  # Convert one-hot encoded y_train to 1D array


#Model evaluation
#from preprolib.myfunctions import evaluate_model

#evaluate_model(rf_model,X_test_transformed, y_test)



In [ ]:
def evaluate_model(curr_model, X_test, y_test):
    import pandas as pd
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    #Convert the one-hot encoded y_test to a 1-dimensional array
    y_test = y_test

    predictions = curr_model.predict(X_test)


    #Calculate the necessary metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    #Print the results for the current model
    print("Model Evaluation:")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

evaluate_model(rf_model,X_test_transformed, y_test)


In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation with 5 folds (you can adjust the number of folds as needed)

cv_scores = cross_val_score(rf_model, X_train_transformed, y_train, cv=5, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation Scores:", cv_scores)
print("Average Cross-validation Accuracy:", cv_scores.mean())


In [ ]:
import joblib

# Save the model to a file
joblib.dump(rf_model, 'trained_rf_model.pkl')

print("Model saved successfully.")



In [ ]:
# Load the model from the file
loaded_rf_model = joblib.load('trained_rf_model.pkl')

# Transform the test data using the preprocessor
df_test_transformed = preprocessor.transform(df_test)

# Make predictions on the test data using the loaded model
test_pred = loaded_rf_model.predict(df_test_transformed)

# Reshape the test_pred array to a 2D array
test_pred_2d = test_pred.reshape(-1, 1)

# Map the predicted values back to their original labels
predicted_labels = label_encoder.inverse_transform(test_pred_2d)

# Print the predicted labels
print("Predicted Labels:")
print(predicted_labels)



In [ ]:
# Create a new DataFrame with the predicted labels
predicted_df = pd.DataFrame(predicted_labels, columns=['Predicted_Customer_Status'])

# Concatenate the predicted_df with df_test
df_test_with_predictions = pd.concat([df_test, predicted_df], axis=1)

# Print the DataFrame with the predicted labels
print(df_test_with_predictions)
